In [50]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import os
from linkedin_jobs_scraper.utils.chrome_driver import build_driver
import os
import logging
import requests
from bs4 import BeautifulSoup
import pandas as pd
cookie = os.environ['LI_AT_COOKIE']
# company_job_link = "https://www.linkedin.com/company/triarq-health/jobs/"

In [29]:

def get_linkedin_company_url(company_name):
    search_query = f"{company_name} LinkedIn"
    google_search_url = f"https://www.google.com/search?q={search_query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    response = requests.get(google_search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    for link in soup.find_all('a'):
        url = link.get('href')
        if url is None:
            continue
        if 'linkedin.com/company' in url:
            linkedin_company_name = url.split('/')[-1]
            company_link = url
            output = {'company_name': company_name,
                'company_linkedin_name':linkedin_company_name, 
                'company_page_url':company_link}

            return output
    
    return "LinkedIn company page not found"

company_name = "Menlo Ventures"
company_data = get_linkedin_company_url(company_name)
company_job_link = f"{company_data['company_page_url']}/jobs/"
print(f"LinkedIn URL for {company_data['company_name']}: {company_data['company_page_url']}")


LinkedIn URL for Menlo Ventures: https://www.linkedin.com/company/menlo-ventures


In [48]:
def get_jobs_page_link(company_job_link):
    driver = build_driver(headless=True)
    driver.get('https://www.linkedin.com')
    driver.add_cookie({
                        'name': 'li_at',
                        'value': os.environ['LI_AT_COOKIE'],
                        'domain': '.www.linkedin.com'
                    })

    driver.get(company_job_link)
    # Wait for the page to load (optional)
    time.sleep(2)
    try:
        job_card = driver.find_element(By.XPATH, "//div[contains(@class, 'org-jobs-recently-posted-jobs-module__show-all-jobs-btn')]")
        link_botton = job_card.find_element(By.XPATH, ".//a")
        link_to_job_page = link_botton.get_attribute("href")
        print(f"msg:found job posted {link_to_job_page}")
        driver.quit()
        return link_to_job_page
    except:
        try:
            no_jobs_element = driver.find_element(By.XPATH, "//div[contains(@class, 'org-jobs-empty-jobs-module artdeco-card')]")
            print(f"msg:no job posted")
            driver.quit()
            return None
        except:
            print(f"msg:unknown error from {company_job_link}")
            driver.quit()
            return None
    
company_name = "Menlo Ventures"
company_data = get_linkedin_company_url(company_name)
company_job = f"{company_data['company_page_url']}/jobs/"
job_page_link = get_jobs_page_link(company_job_link)

msg:found job posted https://www.linkedin.com/jobs/search/?f_C=39208&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3901952996%2C3928435903%2C3822624167%2C3928437664%2C3881990084%2C3921680768%2C3921683412


'https://www.linkedin.com/jobs/search/?f_C=39208&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3901952996%2C3928435903%2C3822624167%2C3928437664%2C3881990084%2C3921680768%2C3921683412'

In [54]:
rollup = pd.read_csv('Rollup.csv')
rollup

FileNotFoundError: [Errno 2] No such file or directory: 'Rollup.csv'

In [ ]:
company_name = "Menlo Ventures"
company_data = get_linkedin_company_url(company_name)
company_job_link = f"{company_data['company_page_url']}/jobs/"